This notebook shows primary color analysis of color image using K-Means algorithm.
The output are N primary colors and their corresponding percentage.

In [ ]:
from ammico.expressions import analyze_images
from ammico.display import show_piechart

In [ ]:
image_paths = [
    "./../ammico/test/data/IMG_3755.jpg",
    "./../ammico/test/data/IMG_2746.png",
    "./../ammico/test/data/IMG_3758.png",
]

In [ ]:
df = analyze_images(image_paths, reduce_colors=True, n_colors=100)

In [ ]:
show_piechart(df)